<a href="https://colab.research.google.com/github/diegogonola/03MIAR---Algoritmos_Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2%2Cno_borrar).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Diego Gonzalez Olabarria  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1Yt5f4sixHQyoSc9cuuxvKO_8MMC5gE-S#scrollTo=HZz52wOppp-U <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.







                                        

# **Modelo**

## **Espacio de soluciones**
El espacio de soluciones es todas las maneras posibles de agrupar las 30 tomas en días, con la restricción de que cada día solo pueden hacerse como maximo 6 tomas.

$$
t1+t2+⋯+tx=30
$$

donde:\
 $ti$ es el número de tomas en el día $i$, $0≤ti≤6$\
 y $x$ es el número de días necesitados, $5≤x≤30$

---

## **Función objetivo**
La funcion objetivo es el minimo de número de viajes de los actores al estudio.

$$
\min \sum_{a \in A} \sum_{d \in D} Y_{ad}
$$

Donde: A es el conjunto de actores (10), D es el conjunto de días de grabación, y $Y_{ad} = 1$ si el actor a trabaja el día d, $Y_{ad} = 0$ si no trabaja ese día.

---

## **Restricciones**

**1**- Cada toma debe ser asignada a un único día.

**2**- Cada día puede tener como máximo 6 tomas.

**3**- Si un actor participa en al menos una toma en un día, debe hacer un viaje.

In [4]:
import pandas as pd
import numpy as np


df = pd.read_csv('Datos_doblaje.csv', skiprows=[0,32],  usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12])
# Solo tomamos las filas y las columnas de interes

display(df.head(33))

df2 = df.copy()
df2 = df2.drop([30])
df2 = df2.drop(["Toma","Total"], axis=1)
# Para quitar ademas las filas y las columnas de los totales, estan bien como orientacion pero traeran problemas en los calculos

def agrupar_tomas_min_viajes(df2, max_tomas_por_dia=6):
    tomas_disponibles = set(df2.index)
    actores_viajes = {actor: 0 for actor in df2.columns}
    dias = []

    while tomas_disponibles:
        dia_actual = []
        actores_presentes = set()

        mejor_toma = None
        max_actores = -1

        # Para la primera toma del día
        for toma in tomas_disponibles:
            num_actores = df2.loc[toma].sum()
            if num_actores > max_actores:
                max_actores = num_actores
                mejor_toma = toma

        dia_actual.append(mejor_toma)
        actores_presentes.update(df2.loc[mejor_toma][df2.loc[mejor_toma] == 1].index)
        tomas_disponibles.remove(mejor_toma)

        # Para encontrar más tomas de ese día
        while len(dia_actual) < max_tomas_por_dia and tomas_disponibles:
            mejor_toma_extra = None
            max_actores_compartidos = 0

            # Se buscan las tomas con más actores en común
            for toma in tomas_disponibles:
                actores_toma = set(df2.loc[toma][df2.loc[toma] == 1].index)
                actores_en_comun = len(actores_presentes.intersection(actores_toma))

                if actores_en_comun > max_actores_compartidos:
                    max_actores_compartidos = actores_en_comun
                    mejor_toma_extra = toma

            # Se actualizan las tomas del dia con la siguiente mejor
            if mejor_toma_extra:
                dia_actual.append(mejor_toma_extra)
                actores_presentes.update(df2.loc[mejor_toma_extra][df2.loc[mejor_toma_extra] == 1].index)
                tomas_disponibles.remove(mejor_toma_extra)
            else:
                break

        for actor in actores_presentes:
            actores_viajes[actor] += 1

        dias.append(dia_actual)

    return dias, actores_viajes

dias_de_grabacion, viajes_por_actor = agrupar_tomas_min_viajes(df2)
viajes = sum(viajes_por_actor.values())

print("Distribución de tomas por día:")
contador = 1
for dia in dias_de_grabacion:
    print("Día", contador, ":", dia)
    contador += 1
print("Total de viajes:", viajes)

,Toma,1,2,3,4,5,6,7,8,9,10,Total
0,1,1,1,1,1,1,0,0,0,0,0,5.0
1,2,0,0,1,1,1,0,0,0,0,0,3.0
2,3,0,1,0,0,1,0,1,0,0,0,3.0
3,4,1,1,0,0,0,0,1,1,0,0,4.0
4,5,0,1,0,1,0,0,0,1,0,0,3.0
5,6,1,1,0,1,1,0,0,0,0,0,4.0
6,7,1,1,0,1,1,0,0,0,0,0,4.0
7,8,1,1,0,0,0,1,0,0,0,0,3.0
8,9,1,1,0,1,0,0,0,0,0,0,3.0
9,10,1,1,0,0,0,1,0,0,1,0,4.0


Distribución de tomas por día:
Día 1 : [0, 5, 6, 10, 11, 19]
Día 2 : [3, 14, 2, 4, 8, 12]
Día 3 : [9, 7, 13, 21, 24, 25]
Día 4 : [1, 26, 15, 16, 18, 22]
Día 5 : [28, 17, 23, 20, 27, 29]
Total de viajes: 32


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

<!-- El problema tiene una complejidad de $O(n^2)$
El programa tiene un bucle principal (while tomas_disponibles:) -> O(T)
  Dentro de este hay un (for toma in tomas_disponibles:) -> O(T) -->

Crear `tomas_disponibles`, `actores_viajes` -> $O(t + a)$

Bucle principal (`while tomas_disponibles:`) -> $O(t)$

>>Para cada toma restante, verificamos qué actores comparten `for toma in tomas_disponibles:` y `num_actores = df2.loc[toma].sum()` -> $O(t·a)$
\
>>Esto se hace `while len(dia_actual) < max_tomas_por_dia`, es decir 6 veces, por lo que no aumenta significativamente la complejidad


>>Actualizar estructuras, agregar tomas al día y actualizar `actores_presentes ` -> $O(t)$

La complejidad final es -> $O(t^2·a)$

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Utiliza una tecnica voraz, porque cada paso busca la solución más óptima, hacer las tomas con mas actores involucrados y luego hacer las tomas de los actores ya presentes. Sin embargo no hay backtraking y puede que no considere las soluciones optimas. Aun así es un algoritmo bastante eficiente y rapido de ejecutar, es por eso que he decidido utilizar este algoritmo.